In [101]:
import matplotlib.pyplot as plt
import numpy as np
import torch

import kornia.io.io as kio
import kornia.augmentation as K
import kornia.enhance as kenh
import torch.nn as nn
import random
import os


In [102]:
# Script to copy images to this directory
"""import os

for file in os.listdir("D:\Vision Lab\lab\droneprojectworkspace\sunum\yolo"):
    if not file.count("pred"):
        os.system(f"copy \"D:\Vision Lab\lab\droneprojectworkspace\sunum\yolo\{file}\" \"images/{file}\"")"""


'import os\n\nfor file in os.listdir("D:\\Vision Lab\\lab\\droneprojectworkspace\\sunum\\yolo"):\n    if not file.count("pred"):\n        os.system(f"copy "D:\\Vision Lab\\lab\\droneprojectworkspace\\sunum\\yolo\\{file}" "images/{file}"")'

In [103]:
# Converts C x H x W to H x W x C
def to_hwc(img: torch.Tensor):
    return img.permute(1, 2, 0).int()


In [104]:
# Load image (C x H x W): https://kornia.readthedocs.io/en/latest/io.html
imgs = [kio.load_image(f"images/{file}", kio.ImageLoadType.UNCHANGED).float() / 255 for file in os.listdir("images")]
img = imgs[4]

In [105]:
from math import pi
from random import shuffle


def random_factor():
    return random.random()

def random_hue_factor():
    factor = random.random() # [0, 1)
    factor -= 0.5 # [-0.5, 0.5)
    factor *= 2 # [-1, 1)
    factor *= pi # [-pi, pi]
    assert -pi <= factor <= pi
    return factor
    
def apply():
    return random.randint(0, 1)

def apply_random_adjustment(img: torch.Tensor):
    # assert torch.Tensor(0) <= img.min() and img.max <= torch.Tensor(1)
    adjusters = [
        # kenh.AdjustBrightness(random_factor() / 2), # https://kornia.readthedocs.io/en/latest/enhance.html#kornia.enhance.AdjustBrightness
        kenh.AdjustContrast(random_factor()), # https://kornia.readthedocs.io/en/latest/enhance.html#kornia.enhance.AdjustContrast
        kenh.AdjustGamma(1.5 * random_factor()), # https://kornia.readthedocs.io/en/latest/enhance.html#kornia.enhance.AdjustGamma
        kenh.AdjustHue(random_hue_factor() * 0.95), # https://kornia.readthedocs.io/en/latest/enhance.html#kornia.enhance.AdjustHue
        kenh.AdjustSaturation(1.5 * random_factor()), # https://kornia.readthedocs.io/en/latest/enhance.html#kornia.enhance.AdjustSaturation
        kenh.Invert(),
    ]

    shuffle(adjusters)
    
    res = torch.Tensor(img)
    for adj in adjusters:
        if apply():
            res = adj(res)
            res += res.min()
            res /= res.max()
    return res


In [ ]:
rows = 5
cols = 5
for i, img in enumerate(imgs):
    fig, axes = plt.subplots(rows, cols)
    fig.set_dpi(1000)
    for r in range(rows):
        for c in range(cols):
            adjusted = apply_random_adjustment(img)
            adjusted *= 255
            adjusted = to_hwc(adjusted)
            axes[r][c].imshow(adjusted)
            axes[r][c].get_xaxis().set_visible(False)
            axes[r][c].get_yaxis().set_visible(False)
    # plt.show()
    plt.savefig(f"augmented/{i}.png");
    print(f"{i} of {len(imgs)} completed")